In [ ]:
import os

import holoviews as hv
import hvplot.xarray  # noqa: F401
import numpy as np
import xarray as xr
from fsspec.implementations.http import HTTPFileSystem

from emit_tools import emit_xarray
from utils import band_index, gamma_adjust, get_rgb_dataset, hv_to_rio_geometry

In [ ]:
# Generate a user token and set it as an environment variable
# Tokens can be generated at https://urs.earthdata.nasa.gov/users/<username>/user_tokens
token = os.environ.get('EARTHDATA_TOKEN')
if token is None:
    raise ValueError('Please set the EARTHDATA_TOKEN environment variable')

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))
# Clean up empty bands.. todo: remove the empty bands after.
ds = ds.fillna(np.nan).where(ds.reflectance!=-0.01)
ds

In [ ]:
%%capture --no-stdout
%autoreload 2

from utils import get_rgb_dataset


# Select wavelengths to be displayed
r = 2100
g = 950
b = 490

# select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.4

## End configuration area ##

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], brightness)


In [ ]:
from holoviews import opts
from holoviews.plotting.links import DataLink

# ten unique colours:
colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# RGB image/map
map = ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

# Set up a holoviews points array to enable plotting of the clicked points
xmid = ds.longitude.values[int(len(ds.longitude) / 2)]
ymid = ds.latitude.values[int(len(ds.latitude) / 2)]

first_polygon = {
    'x': [
        xmid - 0.1,
        xmid + 0.1,
        xmid + 0.1,
        xmid - 0.1,

    ],
    'y': [
        ymid - 0.1,
        ymid - 0.1,
        ymid + 0.1,
        ymid - 0.1,
    ],
    'holes': [[]],
    'id': 0

}
polygons = hv.Polygons(first_polygon, vdims='id')
polygons_stream = hv.streams.PolyDraw(
    data=polygons.columns(), source=polygons, num_objects=1
)

# Function to build spectral plot of clicked location to show on hover stream plot
def polygon_spectra(data):
    print(data)
    geometries = [hv_to_rio_geometry(data)]
    plots = []
    for i, geometry in enumerate(geometries):
        print("Working on one geometry")
        data = ds.reflectance.rio.clip(geometry, drop=False)
        hv_data = hv.Dataset(data, kdims=["bands", "latitude", "longitude"], vdims=["reflectance"])
        agg = hv_data.aggregate("bands", np.nanmean, spreadfn=np.nanstd)

        plots.append(
            (hv.Spread(agg) * hv.Curve(agg))
        )
        polygons_stream.data["id"][i] = i
    return hv.Overlay(plots)

# def update_labels(data):
#     label_points = None
#     if data is not None:
#         label_data = (list(data["x"]), list(data["y"]), [i for i in range(len(data["x"]))])
#         label_points = hv.Points(label_data, vdims='id')
#     return hv.Labels(label_points)

# Define the Dynamic Maps
click_dmap = hv.DynamicMap(polygon_spectra, streams=[polygons_stream])
# label_dmap = hv.DynamicMap(update_labels, streams=[points_stream])

# Plot the Map and Dynamic Map side by side
hv.Layout(map * polygons + click_dmap).cols(1).opts(
    opts.Overlay(show_legend=True, show_title=False, frame_width=600)
)